# Parent-Child Chunk 를 OpenSearch 에 저장

이 노트북은 전체 문서를 Parent, Child 문서로 구분을 하여 오픈 서치 벡터스토어에 저장을 하고, 테스트를 하는 노트북 입니다. 또한 일부 문서로 검증 인덱스를 생성하여 테스트 용으로 만들기 위한 노트북 입니다.

---

## [중요] 사전 실행 노트북
이 노트북은 아래 두개의 셋업 노트북이 먼저 실행이 되어야 합니다.
- (1) Setup 노트북
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup.ipynb 와 같습니다.
    -  [Setup Notebook](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup.ipynb)
- (2) Amazon OpenSearch 설치 노트북    
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb 와 같습니다.
    - [Setup OpenSearch](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb)
    - 위 노트북을 통하여 아래와 같은 정보를 이 노트북에서 사용합니다.
        - opensearch_domain_endpoint
        - opensearch_user_id
        - opensearch_user_password

## [중요] 노트북 생성 결과 
이 노트북을 실행을 하고 나면 Parameter Store 에 두개의 오픈 서치 인덱스 이름이 저장 됩니다. 다음 노트북에서 아래의 변수 이름으로 값을 찾아서 사용하게 됩니다.
- opensearch_index_name
- opensearch_evaluation_index_name

# 1. Bedrock Client 생성

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)


python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot']


In [2]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from local_utils import bedrock, print_ww
from local_utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-west-2
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


# 2. Embedding 모델 로딩

## Embedding Model 선택

In [3]:
Use_Titan_Embedding = True
Use_Cohere_English_Embedding = False

## Embedding Model 로딩

In [4]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

if Use_Titan_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "amazon.titan-embed-text-v1")
    dimension = 1536
elif Use_Cohere_English_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "cohere.embed-english-v3")    
    dimension = 1024
else:
    lim_emb = None

llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f58ed9d09d0>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None, normalize=False)

# 3. Load all Json files

In [5]:
from local_utils.proc_docs import get_load_json, show_doc_json

In [6]:
import glob

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/preprocessed_json/all_processed_data.json'

# List all .txt files in the specified folder
json_files = glob.glob(folder_path)
# json_files = ['data/poc/customer_EFOTA.json']

# Load each item per json file and append to a list
doc_json_list = []
for file_path in json_files:
    doc_json = get_load_json(file_path)
    doc_json_list.append(doc_json)

print("all json files: ", len(doc_json_list))    
# Flatten the list of lists into a single list
all_docs = []
for item in doc_json_list:
        all_docs.extend(item)
        
print("all items: ", len(all_docs))

.[]
all json files:  1
all items:  1732


# 4. Index 생성

## Index 이름 결정

In [7]:
index_name = "<Type index name>"
index_name = "v01-genai-poc-parent-doc-retriever"

## Index 스키마 정의

In [8]:
index_body = {
    'settings': {
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                               'source' : {'type': 'keyword'},
                               'family_tree' : {'type': 'keyword'},                                        
                               'parent_id' : {'type': 'keyword'},                    
                               'last_updated': {'type': 'date'},
                               'project': {'type': 'keyword'},
                               'seq_num': {'type': 'long'},
                               'title': {'type': 'text'},  # For full-text search
                               'url': {'type': 'text'},  # For full-text search
                            }
            },            
            'text': {
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': f"{dimension}"  # Replace with your vector dimension
            }
        }
    }
}


# 5. LangChain OpenSearch VectorStore 생성 
## 선수 조건


## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [9]:
from local_utils.proc_docs import get_parameter

In [10]:
import boto3
region = boto3.Session().region_name
ssm = boto3.client('ssm', region)


opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_user_id',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_user_password',
)


In [11]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

## OpenSearch Client 생성

In [12]:
from local_utils.opensearch import opensearch_utils

In [13]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [14]:
from local_utils.opensearch import opensearch_utils

In [15]:

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=v01-genai-poc-parent-doc-retriever, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v01-genai-poc-parent-doc-retriever'}
Index is created
{'v01-genai-poc-parent-doc-retriever': {'aliases': {},
                                        'mappings': {'properties': {'metadata': {'properties': {'family_tree': {'type': 'keyword'},
                                                                                                'last_updated': {'type': 'date'},
                                                                                                'parent_id': {'type': 'keyword'},
                                                                                                'project': {'type': 'keyword'},
                                                                                                'seq_num': {'type': 'long'},
                                                                                                'source': {

## [중요] Parameter Store 에 index name 저장
- opensearch_index_name 변수 값을 추후 노트북에서 사용합니다.

In [16]:
from local_utils.proc_docs import put_parameter

put_parameter(
    boto3_clinet = ssm, 
    parameter_name = "opensearch_index_name",
    parameter_value = f'{index_name}'
)

Parameter stored successfully.
{'Version': 1, 'Tier': 'Standard', 'ResponseMetadata': {'RequestId': '7b728ae3-81ad-4661-84a7-cb79bd8d9c96', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sun, 10 Mar 2024 00:21:53 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '31', 'connection': 'keep-alive', 'x-amzn-requestid': '7b728ae3-81ad-4661-84a7-cb79bd8d9c96'}, 'RetryAttempts': 0}}


In [17]:
opensearch_index_name = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_index_name',
)

print ("opensearch_index_name: ", opensearch_index_name)



opensearch_index_name:  v01-genai-poc-parent-doc-retriever


## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [18]:
from langchain.vectorstores import OpenSearchVectorSearch

In [19]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

# 6. Chunking JSON Doc 

## Chunk Size and Chunk Overlap Size 결정

In [20]:
parent_chunk_size = 4096
parent_chunk_overlap = 0

child_chunk_size = 1024
child_chunk_overlap = 256

opensearch_parent_key_name = "parent_id"
opensearch_family_tree_key_name = "family_tree"


In [21]:
from local_utils.proc_docs import create_parent_chunk, create_child_chunk

## Parent Chunking

create_parent_chunk() 아래와 같은 작업을 합니다.
- all_docs 에 있는 문서를 parent_chunk_size 만큼으로 청킹 합니다.
- Parent Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: parent
    - parent_id : None

In [22]:
parent_chunk_docs = create_parent_chunk(all_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


Number of parent_chunk_docs= 2325


In [23]:
parent_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'parent', 'parent_id': None})]

### OpenSearch 에 parent chunk 삽입
- 아래는 약 6분 정도 걸립니다.

In [24]:
%%time

parent_ids = vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



CPU times: user 8.11 s, sys: 454 ms, total: 8.57 s
Wall time: 5min 39s


In [25]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 2325, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


삽입된 Parent Chunk 의 첫번째를 확인 합니다. family_tree, parent_id 의 값을 확인 하세요.

In [26]:
def show_opensearch_doc_info(response):
    print("opensearch document id:" , response["_id"])
    print("family_tree:" , response["_source"]["metadata"]["family_tree"])
    print("parent document id:" , response["_source"]["metadata"]["parent_id"])
    print("parent document text: \n" , response["_source"]["text"])

response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: 1f86d9f5-9c84-4e99-a3be-1fe313975a57
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


## Child Chunking

### Child Chunk 생성

아래의 create_child_chunk() 는 다음과 같은 작업을 합니다.
- parent_chunk_docs 각각에 대해서 Child Chunk 를 생성 합니다. 
- Child Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: child
    - parent_id : parent 에 대한 OpenSearch document id

In [27]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


Number of child_chunk_docs= 6884


### 생성된 Child 와 이에 대한 Parent 정보 확인

Child Chunk 한개에 대한 정보를 확인 합니다.

In [28]:
child_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'child', 'parent_id': '1f86d9f5-9c84-4e99-a3be-1fe313975a57'})]

Child 에 대한 Parent 정보 확인

In [29]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = index_name)
show_opensearch_doc_info(response)    


child's parent_id:  1f86d9f5-9c84-4e99-a3be-1fe313975a57

###### Search parent in OpenSearch
opensearch document id: 1f86d9f5-9c84-4e99-a3be-1fe313975a57
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


### OpenSearch 에 Child 삽입
- 아래는 약 13분 정도 소요 됩니다.

In [30]:
%%time

child_ids = vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

length of child_ids:  6884
CPU times: user 24.1 s, sys: 1.33 s, total: 25.4 s
Wall time: 11min 59s


In [31]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 9209, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [32]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: 248c8602-cefe-4e06-be46-d1c3168de53c
family_tree: child
parent document id: 1f86d9f5-9c84-4e99-a3be-1fe313975a57
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


# 7. 검색 테스트

In [33]:
from local_utils.rag import retriever_utils
from local_utils.rag import show_context_used
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### LLM 선택

In [34]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f58ed9d09d0>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f58a41927a0>])

### QA prompt and chain

In [35]:
prompt_template = """

Human: Here is the context, inside <context></context> XML tags.

<context>
{context}
</context>

Only using the context as above, answer the following question with the rules as below:
    - Don't insert XML tag such as <context> and </context> when answering.
    - Write as much as you can
    - Be courteous and polite
    - Only answer the question if you can find the answer in the context with certainty.

Question:
{question}

If the answer is not in the context, just say "For given contexts,I can't find answer."


Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [36]:
chain = load_qa_chain(
    llm=llm_text,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

## Lexical 검색

In [37]:
query = "vidio max size?"

search_filter=[
    #{"term": {"metadata.source": "신한은행"}},
    #{"term": {"metadata.type": "인터넷뱅킹"}},
]

In [38]:
similar_docs_lexical = retriever_utils.get_lexical_similar_docs(
    index_name=index_name,
    os_client=os_client,
    query=query,
    k=5,
    filter=search_filter
)
show_context_used(similar_docs_lexical)

-----------------------------------------------
1. Chunk: 1008 Characters
-----------------------------------------------
. Invalid APK version name. Invalid APK version name. Please change the version name and try
uploading again. 4090001 File too large. Max 4000 devices allowed per upload. Try splitting the
device upload list to reduce the size of .csv you are uploading. Note that there is a max of 4000
devices allowed per upload. 4150000 File type is not supported. The file you are trying to upload is
not supported. Knox Configure only accepts .csv files. App name cannot contain special characters.
Rename the app so it does not contain any special or forbidden characters. App version is invalid.
An app you are trying to upload is invalid. Verify that the version you're attempting to upload is
correct. Try to re-upload the app again. CSV file is required. A system error occurred. Verify that
you've attached a .csv file, try to re-attach the .csv and upload it again. If the problem pe

In [41]:
chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['context', 'question'], template='\n\nHuman: Here is the context, inside <context></context> XML tags.\n\n<context>\n{context}\n</context>\n\nOnly using the context as above, answer the following question with the rules as below:\n    - Don\'t insert XML tag such as <context> and </context> when answering.\n    - Write as much as you can\n    - Be courteous and polite\n    - Only answer the question if you can find the answer in the context with certainty.\n\nQuestion:\n{question}\n\nIf the answer is not in the context, just say "For given contexts,I can\'t find answer."\n\n\nAssistant:'), llm=Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f58ed9d09d0>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f58a41927a0>])), document_v

In [39]:
answer = chain.run(
    input_documents=similar_docs_lexical,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: Here is the context, inside <context></context> XML tags.

<context>
. Invalid APK version name. Invalid APK version name. Please change the version name and try uploading again. 4090001 File too large. Max 4000 devices allowed per upload. Try splitting the device upload list to reduce the size of .csv you are uploading. Note that there is a max of 4000 devices allowed per upload. 4150000 File type is not supported. The file you are trying to upload is not supported. Knox Configure only accepts .csv files. App name cannot contain special characters. Rename the app so it does not contain any special or forbidden characters. App version is invalid. An app you are trying to upload is invalid. Verify that the version you're attempting to upload is correct. Try to re-upload the app again. CSV file is required. A system error occurred. Verify that you've attached a .csv file, tr

## 시맨틱 검색

In [42]:
query = "vidio max size?"

search_filter=[
    #{"term": {"metadata.source": "신한은행"}},
    #{"term": {"metadata.type": "인터넷뱅킹"}},
]

In [43]:
similar_docs_semantic = retriever_utils.get_semantic_similar_docs(
    index_name=index_name,
    os_client=os_client,
    llm_emb=llm_emb,
    query=query,
    k=5,
    boolean_filter=search_filter,
    hybrid=False
)
show_context_used(similar_docs_semantic)

-----------------------------------------------
1. Chunk: 457 Characters
-----------------------------------------------
. Rectangular viewfinder width Set the width of the rectangular viewfinder, specified in screen
width percentage. Only active if View finder selection is set to Rectangular. Available values are
0% to 100%. Default value is 90%. Rectangular viewfinder height Set the height of the rectangular
viewfinder specified in screen height percentage. Only active if View finder selection is set to
Rectangular. Available values are 0% to 100%. Default value is 40%.
metadata:
 {'source': 'all_processed_data.json', 'seq_num': 566, 'title': 'Configure scan engine settings',
'url': 'https://docs.samsungknox.com/admin/knox-capture/how-to-guides/configure-scan-engine-
settings', 'project': 'KCAP', 'last_updated': '2023-10-16', 'family_tree': 'child', 'parent_id':
'f144156e-15e9-4d52-a615-4d77a86c8ed4', 'id': '98f6307e-717c-46de-995e-b28382d38d2b'}
-------------------------------------

In [44]:
answer = chain.run(
    input_documents=similar_docs_semantic,
    question=query
)

print("##############################")
print("query: \n", query)
print("answer: \n", answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


Human: Here is the context, inside <context></context> XML tags.

<context>
. Rectangular viewfinder width Set the width of the rectangular viewfinder, specified in screen width percentage. Only active if View finder selection is set to Rectangular. Available values are 0% to 100%. Default value is 90%. Rectangular viewfinder height Set the height of the rectangular viewfinder specified in screen height percentage. Only active if View finder selection is set to Rectangular. Available values are 0% to 100%. Default value is 40%.

. Values Always use native resolution (default) Use custom resolution &mdash; Applies the values specified by the External display width, External display height, External display scale, and Internal display scale sub-policies. If the custom resolution entered is not supported, the display will revert to its native resolution. Chrome OS 99 and higher &gt;

# 완료 하셨습니다. 다음 노트북으로 가세요.

# [옵션]. 검증 인덱스 생성

아래 부분은 전체 문서의 일부(10%) 만을 샘플링하여 검증용으로 사용합니다. 
그래서 검증용 인덱스를 생성 합니다.

## Index 이름 결정

In [45]:
eval_index_name = "<Type Index Name>"
eval_index_name = "v02-genai-poc-knox-eval-parent-doc-retriever"

## 인덱스 이름을 Parameter Store 에 저장

In [46]:
from local_utils.proc_docs import put_parameter

put_parameter(
    boto3_clinet = ssm, 
    parameter_name = "opensearch_evaluation_index_name",
    parameter_value = f'{eval_index_name}'
)

Parameter stored successfully.
{'Version': 1, 'Tier': 'Standard', 'ResponseMetadata': {'RequestId': 'd380d33b-5c57-41c2-bc04-56fe2e4e1762', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sun, 10 Mar 2024 01:07:30 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '31', 'connection': 'keep-alive', 'x-amzn-requestid': 'd380d33b-5c57-41c2-bc04-56fe2e4e1762'}, 'RetryAttempts': 0}}


In [47]:
opensearch_index_name = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'opensearch_evaluation_index_name',
)

print ("opensearch_index_name: ", opensearch_index_name)



opensearch_index_name:  v02-genai-poc-knox-eval-parent-doc-retriever


## Sampling

In [48]:
import random
def get_sampling_doc(seed, ratio, docs):

    random.seed(seed)
    
    eval_docs = docs[:int(len(docs)*ratio)]
    
    return eval_docs
    
eval_docs = get_sampling_doc(seed=200, ratio=0.05, docs= all_docs)
print("eval docs: ", len(eval_docs))
eval_docs[0:2]
    
    

eval docs:  86


[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27'}),
 Document(page_content='Knox E-FOTA. Knox E-FOTA enables enterprise IT admins t

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [49]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    eval_index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        eval_index_name
    )
    
opensearch_utils.create_index(os_client, eval_index_name, index_body)
index_info = os_client.indices.get(index=eval_index_name)
print("Index is created")
pprint(index_info)    

index_name=v02-genai-poc-knox-eval-parent-doc-retriever, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v02-genai-poc-knox-eval-parent-doc-retriever'}
Index is created
{'v02-genai-poc-knox-eval-parent-doc-retriever': {'aliases': {},
                                                  'mappings': {'properties': {'metadata': {'properties': {'family_tree': {'type': 'keyword'},
                                                                                                          'last_updated': {'type': 'date'},
                                                                                                          'parent_id': {'type': 'keyword'},
                                                                                                          'project': {'type': 'keyword'},
                                                                                                          'seq_num': {'type': 'long'},
                           

## 검증 인덱스 연결 오브젝트 생성

In [50]:
eval_vector_db = OpenSearchVectorSearch(
    index_name= eval_index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
eval_vector_db

## Parent Chunking

In [51]:
parent_chunk_docs = create_parent_chunk(eval_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, 
                                        parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


Number of parent_chunk_docs= 107


In [52]:
parent_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'parent', 'parent_id': None})]

In [53]:
%%time

parent_ids = eval_vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



CPU times: user 380 ms, sys: 27.5 ms, total: 407 ms
Wall time: 15.9 s


In [54]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 107, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [55]:
response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = eval_index_name)
show_opensearch_doc_info(response)    

opensearch document id: b8c0d988-a246-4866-af1b-9327b770cdf9
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


## Child Chunking

In [56]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, 
                                      parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


Number of child_chunk_docs= 308


In [57]:
child_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'child', 'parent_id': 'b8c0d988-a246-4866-af1b-9327b770cdf9'})]

In [58]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


child's parent_id:  b8c0d988-a246-4866-af1b-9327b770cdf9

###### Search parent in OpenSearch
opensearch document id: b8c0d988-a246-4866-af1b-9327b770cdf9
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


In [59]:
%%time

child_ids = eval_vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

length of child_ids:  308
CPU times: user 1.12 s, sys: 57.2 ms, total: 1.18 s
Wall time: 33.9 s


In [60]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 415, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [61]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[5], index_name = eval_index_name)
show_opensearch_doc_info(response)    

opensearch document id: cdd65553-2db9-40e1-9c60-082ccdf7e81e
family_tree: child
parent document id: 34cf1636-4a85-4ce8-8f60-19762788e644
parent document text: 
 . Typically, the latest firmware updates are pushed to devices by their service provider through a Samsung Business-To-Consumer (B2C) FOTA server. The problem with this is that the latest firmware isn't always compatible with a company's in-house apps. Without Knox E-FOTA, companies can only address this problem by doing the following: Always update to the latest OS version, Block all OS updates using their EMM Knox E-FOTA"allows admins to select a firmware version to deploy, even if it's not the latest version. Devices are then locked to that version. When admins have performed compatibility testing on a later firmware version, they can then update devices to the tested version. With Knox E-FOTA, businesses push firmware updates from a Samsung Business-To-Business (B2B) FOTA"server There are exceptions, such as AT&T and Verizo

In [62]:
parent_id = response["_source"]["metadata"]["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


child's parent_id:  34cf1636-4a85-4ce8-8f60-19762788e644

###### Search parent in OpenSearch
opensearch document id: 34cf1636-4a85-4ce8-8f60-19762788e644
family_tree: parent
parent document id: None
parent document text: 
 Knox E-FOTA. Knox E-FOTA enables enterprise IT admins to remotely deploy OS versions and security updates to corporate devices without requiring user interaction. Test updates before deployment to verify compatibility between in-house apps and new OS versions, all while increasing the security of enterprise devices by ensuring the latest security patches are deployed on a schedule. Try for free Audience This document is intended for: System Security Architects - Understand how Knox E-FOTA works, and how you can use it to update fleets of enterprise devices. IT Admins - Learn to manage over-the-air updates for enterprise devices. Try the solution Streamline your mobile update workflow using Knox E-FOTA. Enable version control management, and deploy OS updates by setti

## 검색 테스트

In [64]:
q = "'how to use barcode"
query ={'query': 
        {'bool': {'must': 
                  [{'match': 
                    {'text': 
                     {'query': f"{q}", 'minimum_should_match': '0%', 'operator': 'or'}}}], 
                  'filter': {
                    "term": {
                      "metadata.family_tree": "child"
                    }                      
                  }
                 }
        }
       }
pprint(query)

{'query': {'bool': {'filter': {'term': {'metadata.family_tree': 'child'}},
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': "'how to use "
                                                          'barcode'}}}]}}}


In [65]:
response = opensearch_utils.search_document(os_client, query, eval_index_name)
opensearch_utils.parse_keyword_response(response, show_size=5)

# of searched docs:  10
# of display: 5
---------------------
_id in index:  6d3fe7f6-39aa-4245-8b9e-17892fbebc19
4.210667
How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.
{'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-0

In [66]:
eval_vector_db.similarity_search(q, k=7)

[Document(page_content='. Sign in to the Knox Admin Portal. In the left sidebar, click Knox E-FOTA. 2. Go to Devices > All Devices. 3. Click the device ID of the device you want to view. 4. In the Device details popup, click View device log. Tag a device Tagging is useful if you want to label devices and easily search for them in the device list. For example, you can label devices by their business function or by the location of their device users. This section describes how to tag a single device. If you want to tag multiple devices at once with the same tags, see Perform common operations on devices. 1. Sign in to the Knox Admin Portal. In the left sidebar, click Knox E-FOTA. 2. Go to Devices > All Devices. 3. Click the device ID of the device you want to tag. 4. In the Device details dialog, under the Tags field, enter the labels to associate with the device. For example: sales. 5. Click Save.', metadata={'source': 'all_processed_data.json', 'seq_num': 25, 'title': 'Manage devices',